<a href="https://colab.research.google.com/github/JalpaK88/Projects_SQL/blob/main/SQL_bigquery_hackers_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.cloud import bigquery

# Step 1: Authenticate
auth.authenticate_user()

# Step 2: Prompt user for project ID
PROJECT_ID = input("Enter your GCP project ID: ").strip()

# Check if a project ID was entered
if not PROJECT_ID:
    raise ValueError("You must enter a valid GCP project ID!")

# Step 3: Initialize BigQuery client with the provided project ID
client = bigquery.Client(project=PROJECT_ID)
print("Using project:", client.project)

Enter your GCP project ID: brilliant-flame-471022-p3
Using project: brilliant-flame-471022-p3


In [ ]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

In [ ]:
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [ ]:
# List all the tables in the "hacker_news" dataset
tables = list(client.list_tables(dataset))

In [ ]:
# Print names of all tables in the dataset (there are four!)
for table in tables:
  print(table.table_id)

full


In [ ]:
#fetch table
table_ref = dataset_ref.table("full")
# API request
table = client.get_table(table_ref)

In [ ]:
table.schema

[SchemaField('title', 'STRING', 'NULLABLE', None, 'Story title', (), None),
 SchemaField('url', 'STRING', 'NULLABLE', None, 'Story url', (), None),
 SchemaField('text', 'STRING', 'NULLABLE', None, 'Story or comment text', (), None),
 SchemaField('dead', 'BOOLEAN', 'NULLABLE', None, 'Is dead?', (), None),
 SchemaField('by', 'STRING', 'NULLABLE', None, "The username of the item's author.", (), None),
 SchemaField('score', 'INTEGER', 'NULLABLE', None, 'Story score', (), None),
 SchemaField('time', 'INTEGER', 'NULLABLE', None, 'Unix time', (), None),
 SchemaField('timestamp', 'TIMESTAMP', 'NULLABLE', None, 'Timestamp for the unix time', (), None),
 SchemaField('type', 'STRING', 'NULLABLE', None, 'type of details (comment comment_ranking poll story job pollopt)', (), None),
 SchemaField('id', 'INTEGER', 'NULLABLE', None, "The item's unique id.", (), None),
 SchemaField('parent', 'INTEGER', 'NULLABLE', None, 'Parent comment ID', (), None),
 SchemaField('descendants', 'INTEGER', 'NULLABLE', N

In [ ]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,True,None,<NA>,1437994174,2015-07-27 10:49:34+00:00,story,9954404,<NA>,<NA>,<NA>,<NA>
1,None,None,None,<NA>,None,<NA>,1437996330,2015-07-27 11:25:30+00:00,story,9954508,<NA>,<NA>,<NA>,<NA>
2,None,None,None,True,None,<NA>,1437996498,2015-07-27 11:28:18+00:00,story,9954518,<NA>,<NA>,<NA>,<NA>
3,None,None,None,<NA>,None,<NA>,1437997173,2015-07-27 11:39:33+00:00,story,9954559,<NA>,<NA>,<NA>,<NA>
4,None,None,None,<NA>,None,<NA>,1260835350,2009-12-15 00:02:30+00:00,story,995467,<NA>,<NA>,<NA>,<NA>


In [ ]:
# Preview the first five entries in the "by" column of the "full" table
client.list_rows(table, selected_fields=table.schema[:5],max_results=5).to_dataframe()

,title,url,text,dead,by
0,None,None,None,True,None
1,None,None,None,<NA>,None
2,None,None,None,True,None
3,None,None,None,<NA>,None
4,None,None,None,<NA>,None


In [ ]:
# Select only the 5th column
fifth_col = [table.schema[4]]  # schema is zero-indexed

# Query rows with only that column
rows = client.list_rows(table, selected_fields=fifth_col,max_results=5)  # just one SchemaField

df = rows.to_dataframe()
print(df)

     by
0  None
1  None
2  None
3  None
4  None


In [ ]:
# SELECT Query to get the score column from every row where the type column has value "job"
query = """
        SELECT score, title
        FROM `bigquery-public-data.hacker_news.full`
        WHERE type = "job"
        """

# # Create a QueryJobConfig object to estimate size of query without running it
# dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# # API request - dry run query to estimate costs
# dry_run_query_job = client.query(query, job_config=dry_run_config)

# print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

This query will process 708611355 bytes.


In [ ]:
query_job = client.query(query)

In [ ]:
job_score_title = query_job.to_dataframe()

In [ ]:
job_score_title.score.value_counts().head()

,count
score,
1,16558
3,70
4,55
5,52
2,41


In [ ]:
job_score_title.title.value_counts().head()

,count
title,
EasyPost (YC S13) Is Hiring,38
Strikingly (YC W13) is hiring in our Shanghai office,35
Mino Games Is Hiring Programmers in Montreal,33
Scale AI is hiring engineers to accelerate the development of AI,31
Flexport is hiring a senior front-end engineer,29


In [ ]:
# Top 5 scores
print("Top 5 job score:")
print(job_score_title["score"].value_counts().head())

# Top 5 titles in the result
print("\nTop 5 job title:")
print(job_score_title["title"].value_counts().head())

Top 5 job score:
score
1    16558
3       70
4       55
5       52
2       41
Name: count, dtype: Int64

Top 5 job title:
title
EasyPost (YC S13) Is Hiring                                         38
Strikingly (YC W13) is hiring in our Shanghai office                35
Mino Games Is Hiring Programmers in Montreal                        33
Scale AI is hiring engineers to accelerate the development of AI    31
Flexport is hiring a senior front-end engineer                      29
Name: count, dtype: int64


In [ ]:
# Print average score for job posts
job_score_title.score.mean()

np.float64(1.6461484552940489)

In [ ]:
# Group by, Having, Count
query_popular = """
                SELECT parent, COUNT(id)
                FROM `bigquery-public-data.hacker_news.full`
                GROUP BY parent
                HAVING COUNT(id) > 10
                """

In [ ]:
query_job = client.query(query_popular)

In [ ]:
parent_count = query_job.to_dataframe()

In [ ]:
parent_count.head()

,parent,f0_
0,9906107,47
1,9085695,43
2,9158913,61
3,9191248,76
4,7880676,75


In [ ]:
query_popular_2 = """
              SELECT parent, count(id) AS Numposts
              FROM `bigquery-public-data.hacker_news.full`
              GROUP BY parent
              HAVING Numposts > 10
              """
query_good = """
              SELECT parent, count(id)
              FROM `bigquery-public-data.hacker_news.full`
              GROUP BY parent

              """

In [ ]:
# query_job = client.query(query_popular_2)
query_job = client.query(query_good)

In [ ]:
parent_count = query_job.to_dataframe()

In [ ]:
parent_count.head()

,parent,f0_
0,38671864,9
1,19540979,8
2,30171659,9
3,30732274,24
4,4196113,8


In [ ]:
# Hacker News would like to send awards to everyone who has written more than 10,000 posts.
# Write a query that returns all authors with more than 10,000 posts as well as their post counts. Call the column with post counts NumPosts
query_popular_3 = """
              SELECT `by` AS author, COUNT(1) AS NumPosts
              FROM `bigquery-public-data.hacker_news.full`
              GROUP BY author
              HAVING COUNT(1) > 10000
"""

# prolific_commenters_query = """
#                             SELECT `by` AS author, COUNT(1) AS NumPosts
#                             FROM `bigquery-public-data.hacker_news.full`
#                             GROUP BY author
#                             HAVING COUNT(1) > 10000
#                             """

In [ ]:
query_job = client.query(query_popular_3)

In [ ]:
author_count = query_job.to_dataframe()


In [ ]:
author_count

,author,NumPosts
0,Retric,28748
1,ams6110,11149
2,AnthonyMouse,14705
3,mikeash,21188
4,dllthomas,10744
...,...,...
210,jessaustin,14209
211,tyingq,17122
212,vkou,11457
213,criddell,10147


In [ ]:
# How many comments have been deleted?
# (If a comment was deleted, the deleted column in the table will have the value True.)

query_deleted = """
                SELECT Count(id) As NumDeletedPosts
                FROM `bigquery-public-data.hacker_news.full`
                WHERE deleted = True
"""


In [ ]:
query_job = client.query(query_deleted)

In [ ]:
deleted_count = query_job.to_dataframe()

In [ ]:
deleted_count

,NumDeletedPosts
0,0
